In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

In [ ]:
from toolbox_continu_inzicht import Config, DataAdapter

In [ ]:
path = Path.cwd() / "data_sets"
config = Config(config_path=path / "integrate_statistics.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
from toolbox_continu_inzicht.fragility_curves import IntegrateFragilityCurve

In [ ]:
integrate_statistics_per_section = IntegrateFragilityCurve(data_adapter=data_adapter)
integrate_statistics_per_section.run(
    input=["exceedance_curve_csv", "fragility_curve_csv"], output="result"
)

In [ ]:
initial_integrated_curve = integrate_statistics_per_section.df_out
fig, ax = plt.subplots()
ax.bar(
    initial_integrated_curve["waterlevels"],
    initial_integrated_curve["probability_contribution"],
    width=0.1,
)
ax.set_xlim([0, 10])

In [ ]:
from toolbox_continu_inzicht.base.fragility_curve import FragilityCurve

In [ ]:
fragility_curve = FragilityCurve(data_adapter=data_adapter)

In [ ]:
fragility_curve.load("fragility_curve_csv")

In [ ]:
first_fragility_curve = fragility_curve.df_out

In [ ]:
update_level = 4.3
trust_factor = 1
fragility_curve.reliability_update(update_level=update_level, trust_factor=trust_factor)

In [ ]:
updated_curve = fragility_curve.df_out

In [ ]:
fig, ax = plt.subplots(1, 1)
first_fragility_curve.plot(x="waterlevels", y="failure_probability", ax=ax)
updated_curve.plot(x="waterlevels", y="failure_probability", ax=ax, ls="--")

In [ ]:
data_adapter.set_dataframe_adapter(
    "updated_fragility_curve", fragility_curve.df_out, if_not_exist="create"
)

In [ ]:
integrate_statistics_per_section = IntegrateFragilityCurve(data_adapter=data_adapter)
integrate_statistics_per_section.run(
    input=["exceedance_curve_csv", "updated_fragility_curve"], output="result"
)

In [ ]:
df_out = integrate_statistics_per_section.df_out
fig, ax = plt.subplots()
ax.bar(
    initial_integrated_curve["waterlevels"],
    initial_integrated_curve["probability_contribution"],
    width=0.1,
)
ax.bar(df_out["waterlevels"], df_out["probability_contribution"], width=0.1)
ax.set_xlim([0, 10])
ax.set_xlabel("Water level")
ax.set_ylabel("Probability contribution")

# Repeat but in a widget

In [ ]:
from ipywidgets import interact


@interact(update_level=(0.0, 10.0, 0.1), trust_factor=(0, 1, 0.1))
def make_plot(update_level, trust_factor):
    integrate_statistics_per_section = IntegrateFragilityCurve(
        data_adapter=data_adapter
    )
    integrate_statistics_per_section.run(
        input=["exceedance_curve_csv", "fragility_curve_csv"], output="result"
    )
    fragility_curve = FragilityCurve(data_adapter=data_adapter)
    fragility_curve.load("fragility_curve_csv")
    fragility_curve.reliability_update(
        update_level=update_level, trust_factor=trust_factor
    )
    data_adapter.set_dataframe_adapter(
        "updated_fragility_curve", fragility_curve.df_out, if_not_exist="create"
    )
    integrate_statistics_per_section = IntegrateFragilityCurve(
        data_adapter=data_adapter
    )
    integrate_statistics_per_section.run(
        input=["exceedance_curve_csv", "updated_fragility_curve"], output="result"
    )

    df_out = integrate_statistics_per_section.df_out
    fig, ax = plt.subplots()
    ax.bar(
        initial_integrated_curve["waterlevels"],
        initial_integrated_curve["probability_contribution"],
        width=0.1,
    )
    ax.bar(df_out["waterlevels"], df_out["probability_contribution"], width=0.1)
    ax.set_xlim([0, 10])
    ax.set_xlabel("Water level")
    ax.set_ylabel("Probability contribution")
    ax.set_title(
        "Updated probability contribution with given update level and trust factor"
    )